In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.autograd import Variable
from concrete_dropout import ConcreteDropout
import numpy as np

np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [65]:
from torch.utils.data import DataLoader

In [2]:
import pandas as pd

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [53]:
Y.isnull().any(axis=1).values.shape

((17248,), (17248, 11))

In [69]:
X = pd.read_csv("truth_X_pd.csv")
Y = pd.read_csv("truth_Y_pd.csv")
nan_rows = Y.isnull().any(axis=1).values
X = X.loc[~nan_rows, :]#.values
Y = Y.loc[~nan_rows, :]#.values

In [55]:
class ConcreteDropout(nn.Module):
    def __init__(self, weight_regularizer=1e-6,
                 dropout_regularizer=1e-5, init_min=0.1, init_max=0.1):
        super(ConcreteDropout, self).__init__()

        self.weight_regularizer = weight_regularizer
        self.dropout_regularizer = dropout_regularizer
        
        init_min = np.log(init_min) - np.log(1. - init_min)
        init_max = np.log(init_max) - np.log(1. - init_max)
        
        self.p_logit = nn.Parameter(torch.empty(1).uniform_(init_min, init_max))
        
    def forward(self, x, layer):
        p = torch.sigmoid(self.p_logit)
        
        out = layer(self._concrete_dropout(x, p))
        
        sum_of_square = 0
        for param in layer.parameters():
            sum_of_square += torch.sum(torch.pow(param, 2))
        
        weights_regularizer = self.weight_regularizer * sum_of_square / (1 - p)
        
        dropout_regularizer = p * torch.log(p)
        dropout_regularizer += (1. - p) * torch.log(1. - p)
        
        input_dimensionality = x[0].numel() # Number of elements of first item in batch
        dropout_regularizer *= self.dropout_regularizer * input_dimensionality
        
        regularization = weights_regularizer + dropout_regularizer
        return out, regularization
        
    def _concrete_dropout(self, x, p):
        eps = 1e-7
        temp = 0.1

        unif_noise = torch.rand_like(x)

        drop_prob = (torch.log(p + eps)
                    - torch.log(1 - p + eps)
                    + torch.log(unif_noise + eps)
                    - torch.log(1 - unif_noise + eps))
        
        drop_prob = torch.sigmoid(drop_prob / temp)
        random_tensor = 1 - drop_prob
        retain_prob = 1 - p
        
        x  = torch.mul(x, random_tensor)
        x /= retain_prob
        
        return x

In [56]:
N = int(X.shape[0]*0.9) # Number of data points
nb_epoch = 20
nb_val_size = X.shape[0] - N # Validation size
nb_features = 20 # Hidden layer size
Q = 9 # Data dimensionality
D = 11 # One mean, one log_var
K_test = 20 # Number of MC samples
nb_reps = 3 # Number of times to repeat experiment
batch_size = 10
l = 1e-4 # Lengthscale
nb_val_size, N

(1725, 15519)

In [57]:
class Model(nn.Module):
    def __init__(self, nb_features, weight_regularizer, dropout_regularizer):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(Q, nb_features)
        self.linear2 = nn.Linear(nb_features, nb_features)
        self.linear3 = nn.Linear(nb_features, nb_features)

        self.linear4_mu = nn.Linear(nb_features, D)
        self.linear4_logvar = nn.Linear(nb_features, D)

        self.conc_drop1 = ConcreteDropout(weight_regularizer=weight_regularizer,
                                          dropout_regularizer=dropout_regularizer)
        self.conc_drop2 = ConcreteDropout(weight_regularizer=weight_regularizer,
                                          dropout_regularizer=dropout_regularizer)
        self.conc_drop3 = ConcreteDropout(weight_regularizer=weight_regularizer,
                                          dropout_regularizer=dropout_regularizer)
        self.conc_drop_mu = ConcreteDropout(weight_regularizer=weight_regularizer,
                                             dropout_regularizer=dropout_regularizer)
        self.conc_drop_logvar = ConcreteDropout(weight_regularizer=weight_regularizer,
                                                 dropout_regularizer=dropout_regularizer)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        regularization = torch.empty(5, device=x.device)
        
        x1, regularization[0] = self.conc_drop1(x, nn.Sequential(self.linear1, self.relu))
        x2, regularization[1] = self.conc_drop2(x1, nn.Sequential(self.linear2, self.relu))
        x3, regularization[2] = self.conc_drop3(x2, nn.Sequential(self.linear3, self.relu))

        mean, regularization[3] = self.conc_drop_mu(x3, self.linear4_mu)
        log_var, regularization[4] = self.conc_drop_logvar(x3, self.linear4_logvar)

        return mean, log_var, regularization.sum()

def heteroscedastic_loss(true, mean, log_var):
    precision = torch.exp(-log_var)
    return torch.mean(torch.sum(precision * (true - mean)**2 + log_var, 1), 0)

In [58]:
def fit_model(nb_epoch, X, Y):
    N = X.shape[0]
    wr = l**2. / N
    dr = 2. / N
    model = Model(nb_features, wr, dr)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters())
    
    for i in range(nb_epoch):
        old_batch = 0
        for batch in range(int(np.ceil(X.shape[0]/batch_size))):
            batch = (batch + 1)
            _x = X[old_batch: batch_size*batch]
            _y = Y[old_batch: batch_size*batch]
            
            x = Variable(torch.FloatTensor(_x)).to(device)
            y = Variable(torch.FloatTensor(_y)).to(device)
            
            mean, log_var, regularization = model(x)
                        
            loss = heteroscedastic_loss(y, mean, log_var) + regularization
             
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    return model

In [59]:
def logsumexp(a):
    a_max = a.max(axis=0)
    return np.log(np.sum(np.exp(a - a_max), axis=0)) + a_max

def test(Y_true, K_test, means, logvar):
    """
    Estimate predictive log likelihood:
    log p(y|x, D) = log int p(y|x, w) p(w|D) dw
                 ~= log int p(y|x, w) q(w) dw
                 ~= log 1/K sum p(y|x, w_k) with w_k sim q(w)
                  = LogSumExp log p(y|x, w_k) - log K
    :Y_true: a 2D array of size N x dim
    :MC_samples: a 3D array of size samples K x N x 2*D
    """
    k = K_test
    N = Y_true.shape[0]
    mean = means 
    logvar = logvar
    test_ll = -0.5 * np.exp(-logvar) * (mean - Y_val.squeeze())**2. - 0.5 * logvar - 0.5 * np.log(2 * np.pi) #Y_true[None]
    test_ll = np.sum(np.sum(test_ll, -1), -1)
    test_ll = logsumexp(test_ll) - np.log(k)
    pppp = test_ll / N  # per point predictive probability
    rmse = np.mean((np.mean(mean, 0) - Y_val.squeeze())**2.)**0.5
    return pppp, rmse

In [60]:
def plot(X_train, Y_train, X_val, Y_val, means, param):
    # means ~ [K_test, data_size, num_params]
    # Number of repeated sampling
    K_test = means.shape[0]
    indx = np.argsort(X_val[:, param]) # plt.plot requires x-ordered points
    _, (ax1, ax2, ax3, ax4) = pylab.subplots(1, 4,figsize=(12, 1.5), sharex=True, sharey=True)
    # 1. Show just the train set
    ax1.scatter(X_train[:, param], Y_train[:, param], c='y')
    ax1.set_title('Train set')
    # 1 overlaid with 2. Show predicted means (averaged across samples) for validation set
    ax2.plot(X_val[indx, param], np.mean(means, 0)[indx, param], color='skyblue', lw=3)
    ax2.scatter(X_train[:, param], Y_train[:, 0], c='y')
    ax2.set_title('+Predictive mean')
    # 2 overlaid with 3. All predicted means across samples (not averaged) for validation set
    for i in range(K_test):
        ax3.scatter(X_val[:, param], means[i, :, param], c='b', alpha=0.2, lw=0)
    ax3.plot(X_val[indx, param], np.mean(means, 0)[indx, param], color='skyblue', lw=3)
    ax3.set_title('+MC samples on validation X')
    # Just the validation set
    ax4.scatter(X_val[:, param], Y_val[:, param], c='r', alpha=0.2, lw=0)
    ax4.set_title('Validation set')
    
    pylab.show()

In [61]:
#X, Y = gen_data(N + nb_val_size)
X_train, Y_train = X[:N], Y[:N]
X_val, Y_val = X[N:], Y[N:]

In [62]:
model = fit_model(nb_epoch, X_train, Y_train)

In [63]:
model.eval()

Model(
  (linear1): Linear(in_features=9, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=1024, bias=True)
  (linear3): Linear(in_features=1024, out_features=1024, bias=True)
  (linear4_mu): Linear(in_features=1024, out_features=11, bias=True)
  (linear4_logvar): Linear(in_features=1024, out_features=11, bias=True)
  (conc_drop1): ConcreteDropout()
  (conc_drop2): ConcreteDropout()
  (conc_drop3): ConcreteDropout()
  (conc_drop_mu): ConcreteDropout()
  (conc_drop_logvar): ConcreteDropout()
  (relu): ReLU()
)

In [64]:
MC_samples = [model(Variable(torch.FloatTensor(X_val)).to(device)) for _ in range(K_test)]

In [23]:
MC_samples[0][0].shape, len(MC_samples), X_val.shape[0]

(torch.Size([2566, 8]), 20, 2566)

In [24]:
torch.stack([tup[0] for tup in MC_samples]).shape

torch.Size([20, 2566, 8])

In [21]:
import pylab

In [66]:
means = torch.stack([tup[0] for tup in MC_samples]).view(K_test, X_val.shape[0], D).cpu().data.numpy()
logvar = torch.stack([tup[1] for tup in MC_samples]).view(K_test, X_val.shape[0], D).cpu().data.numpy()
pppp, rmse = test(Y_val, K_test, means, logvar)
param = 4
epistemic_uncertainty = np.var(means, 0).mean(0)[param]
#logvar = np.mean(logvar, 0)[param]
aleatoric_uncertainty = np.exp(logvar).mean(0)
ps = np.array([torch.sigmoid(module.p_logit).cpu().data.numpy()[0] for module in model.modules() if hasattr(module, 'p_logit')])
#plot(X_train, Y_train, X_val, Y_val, means, param=param)

In [67]:
np.save("means_run2", means.reshape(20, -1))
np.save("logvar_run2", logvar.reshape(20, -1))

In [62]:
logvar.shape

(20, 2566, 8)

In [45]:
np.mean(logvar, 0).shape

(8,)